## Upload and Select Stimulus (Video or Image)
Upload the stimulus video or image file.

In [26]:
import os

In [27]:
def create_eye_tracking_video(gaze_df, video_filename="eye_tracking_video.mp4", fps=30, width=1280, height=1024):
    """
    Creates a blank video with a duration matching the eye movement timestamps in the dataset.

    Parameters:
        gaze_df (pandas.DataFrame): The eye movement dataset containing 'time' column.
        video_filename (str): Name of the output video file.
        fps (int): Frames per second.
        width (int): Video width.
        height (int): Video height.
    """
    if "frame_idx" in gaze_df.columns:
        # If 'frame_idx' exists, use its max value to determine total frames
        total_frames = int(gaze_df["frame_idx"].max()) + 1
    else:
        # Otherwise, compute total frames based on timestamps
        min_time = gaze_df["time"].min() / 1000.0  # Convert ms → s
        max_time = gaze_df["time"].max() / 1000.0  # Convert ms → s
        duration = max_time - min_time  # Total video duration in seconds
        total_frames = int(duration * fps)

    # Ensure at least 1 frame is written
    total_frames = max(1, total_frames)

    print(f"Creating video of duration: {duration:.2f} sec ({total_frames} frames)")

    # Set up the video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec
    out = cv2.VideoWriter(video_filename, fourcc, fps, (width, height))

    # Create white frames and write to video
    white_frame = np.ones((height, width, 3), dtype=np.uint8) * 255  # White background
    for _ in range(total_frames):
        out.write(white_frame)

    out.release()
    print(f"Video saved as {video_filename}")

In [28]:
import ipyfilechooser
import cv2
import numpy as np
from IPython.display import display

# Step 1: Create the file chooser widget
base_dir = os.getcwd()  # Start from the current working directory
while not base_dir.endswith("pymovements-videoreplay") and os.path.dirname(base_dir) != base_dir:
    base_dir = os.path.dirname(base_dir)  # Go up one level until we find the project root

default_path = os.path.join(base_dir, "data")

os.makedirs(default_path, exist_ok=True)

chooser = ipyfilechooser.FileChooser(default_path)
chooser.filter_pattern = ['*.mp4', '*.jpg', '*.jpeg', '*.png']  # Show video & image files
display(chooser)

# Step 2: Function to move the selected file or use the generated video
def save_uploaded_stimulus():
    selected_file_path = chooser.selected  # Get selected file path

    if selected_file_path is None:
        # If no video or image is uploaded, generate a white background video
        default_video = "white_background.mp4"
        if not os.path.exists(default_video):
            print(f"No stimulus uploaded. Generating default video with white background: {default_video}")
            create_eye_tracking_video(dataset.gaze[0].frame.to_pandas(), default_video)

        # Check if the generated image exists
        if os.path.exists(default_video):
            print(f"Using generated white background video: {default_video}")
            selected_file_path = default_video
        else:
            print("ERROR: No stimulus uploaded and failed to generate an image. Please upload an image or video.")
            return

    # Verify the file exists
    stimulus_name = os.path.basename(selected_file_path)
    if os.path.exists(selected_file_path):
        print(f"Stimulus saved successfully as '{stimulus_name}'!")
        print(f"File Size: {os.path.getsize(selected_file_path)} bytes")
    else:
        print("ERROR: Stimulus file was not saved correctly!")
        
    return selected_file_path    

FileChooser(path='C:\Users\Kirthan\IdeaProjects\pymovements-videoreplay\data', filename='', title='', show_hid…

In [29]:
# Step 3: After Selecting the File, Run:
selected_file_path = save_uploaded_stimulus()

Stimulus saved successfully as 'reading-genome-2.png'!
File Size: 175729 bytes


## Initialize and Run VideoPlayer
Run the VideoPlayer to visualize eye-tracking data overlay.

In [30]:
import sys
import importlib

# Ensure 'src' is in sys.path
src_path = os.path.abspath(os.path.join('src'))
if src_path not in sys.path:
    sys.path.append(src_path)

# Import the module
import videoreplay.video_player

# Reload the module
importlib.reload(videoreplay.video_player)

# Now import the updated class
from videoreplay.video_player import VideoPlayer

# Initialize the VideoPlayer with the uploaded video
player = VideoPlayer(stimulus_path=selected_file_path,
                     dataset_path="data/SB-SAT/fixation",
                     recording_session="msd001")

Loading gaze data from: data\SB-SAT\fixation\18sat_fixfinal.csv


## Play Video with Gaze Overlay

In [31]:
player.play(speed=5.0)

## Navigate Through Fixations
Use 'n' for next fixation, 'p' for previous, 'q' to quit.

In [32]:
player.fixation_navigation()

## Export Gaze Replay as MP4
Save the replay with gaze overlay.

In [33]:
player.export_replay("gaze_replay_slow", 5)

Exporting gaze replay for an image stimulus...
Image-based replay exported as MP4: gaze_replay_slow.mp4
